# INITIALIZATION

In [1]:
from sys import path
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import pandas as pd
import math

if not ("C:\\Users\\julesl\\Documents (Local)\\JupyterLabWorkspace\\Library\\" in path) :
    path.append("C:\\Users\\julesl\\Documents (Local)\\JupyterLabWorkspace\\Library\\")

from FeedingHabitat.feedingHabitat import FeedingHabitat

fh = FeedingHabitat()

In [2]:
%%time
filepath_directory = "C:\\Users\\julesl\\Documents (Local)\\JupyterLabWorkspace\\\Data\\Concat\\PO\\"
fh.coords = xr.open_dataset(filepath_directory+"MFCat_po_025x30_temperature_L1_1998_2019.nc").coords
fh.layers_number = 3
fh.cohorts_number=1

# OXYGEN
fh.variables_dictionary["oxygen_L1"] = np.nan_to_num(xr.open_dataset(filepath_directory+"MFCat_po_025x30_oxygen_L1_1970_1970.nc")['oxygen_L1'].data)
fh.variables_dictionary["oxygen_L2"] = np.nan_to_num(xr.open_dataset(filepath_directory+"MFCat_po_025x30_oxygen_L2_1970_1970.nc")['oxygen_L2'].data)
fh.variables_dictionary["oxygen_L3"] = np.nan_to_num(xr.open_dataset(filepath_directory+"MFCat_po_025x30_oxygen_L3_1970_1970.nc")['oxygen_L3'].data)

#TEMPERATURE
fh.variables_dictionary["temperature_L1"] = np.nan_to_num(xr.open_dataset(filepath_directory+"MFCat_po_025x30_temperature_L1_1998_2019.nc")['temperature_L1'].data)
fh.variables_dictionary["temperature_L2"] = np.nan_to_num(xr.open_dataset(filepath_directory+"MFCat_po_025x30_temperature_L2_1998_2019.nc")['temperature_L2'].data)
fh.variables_dictionary["temperature_L3"] = np.nan_to_num(xr.open_dataset(filepath_directory+"MFCat_po_025x30_temperature_L3_1998_2019.nc")['temperature_L3'].data)

# FORAGE
fh.variables_dictionary["forage_epi"]     = np.nan_to_num(xr.open_dataset(filepath_directory+"\\forage-vgpm\\Fbiom_epi.nc")['epi'].data)
fh.variables_dictionary["forage_meso"]    = np.nan_to_num(xr.open_dataset(filepath_directory+"\\forage-vgpm\\Fbiom_meso.nc")['meso'].data)
fh.variables_dictionary["forage_mmeso"]   = np.nan_to_num(xr.open_dataset(filepath_directory+"\\forage-vgpm\\Fbiom_mmeso.nc")['mmeso'].data)
fh.variables_dictionary["forage_bathy"]   = np.nan_to_num(xr.open_dataset(filepath_directory+"\\forage-vgpm\\Fbiom_bathy.nc")['bathy'].data)
fh.variables_dictionary["forage_mbathy"]  = np.nan_to_num(xr.open_dataset(filepath_directory+"\\forage-vgpm\\Fbiom_mbathy.nc")['mbathy'].data)
fh.variables_dictionary["forage_hmbathy"] = np.nan_to_num(xr.open_dataset(filepath_directory+"\\forage-vgpm\\Fbiom_hmbathy.nc")['hmbathy'].data)

#L/W
#fh.variables_dictionary['cohorts_mean_weight'] = np.array([0.0003, 0.00109, 0.00571, 0.023, 0.08,  0.18,  0.32,  0.53,  0.78,  1.09,  1.44,  1.84, 2.27 , 2.73 , 3.21 , 3.72,  4.23,  4.76,  5.30,  5.83 , 6.36,  6.89 , 7.40 , 7.91 ,8.41  ,8.89  ,9.36,  9.81, 10.25, 10.66 ,11.07 ,11.45 ,11.82, 12.17, 12.51 ,12.83, 15.56])
#fh.variables_dictionary['cohorts_mean_length'] = np.array([3, 4.5 ,7.52, 11.65, 16.91, 21.83 ,26.43 ,30.72, 34.73 ,38.49, 41.99, 45.27, 48.33 ,51.19, 53.86, 56.36, 58.70, 60.88, 62.92, 64.83, 66.61 ,68.27, 69.83, 71.28, 72.64 ,73.91, 75.10, 76.21, 77.25, 78.22, 79.12, 79.97, 80.76 ,81.50, 82.19, 82.83 ,87.96])
fh.variables_dictionary['cohorts_mean_weight'] = np.array([15.56])
fh.variables_dictionary['cohorts_mean_length'] = np.array([87.96])

#OTHERS
fh.daysLength(xr.open_dataset(filepath_directory+"MFCat_po_025x30_temperature_L1_1998_2019.nc"))
fh.partial_oxygen_time_axis = True
fh.setMask(from_text=filepath_directory+'mask_freeglorys_025deg_corrected_noIO_short.txt',
           expend_time=xr.open_dataset(filepath_directory+"MFCat_po_025x30_temperature_L1_1998_2019.nc").time.size)

Wall time: 49.8 s


In [4]:
for var in fh.variables_dictionary.values() :
    print(var.shape)
print(fh.partial_oxygen_time_axis)
print(fh.cohorts_number)

(264, 480, 808)
(264, 480, 808)
(264, 480, 808)
(264, 480, 808)
(264, 480, 808)
(264, 480, 808)
(264, 480, 808)
(264, 480, 808)
(264, 480, 808)
(12, 480, 808)
(12, 480, 808)
(12, 480, 808)
(264, 480, 808)
(1,)
(1,)
True
1


# Ha COMPUTATION

In [5]:
%load_ext memory_profiler

In [5]:
fpc = [0.04507713477165387,
       0.05,
       4.999999999806264,
       0.0,
       0.0,
       2.556631372954981]

gamma    = 0.0139731044278165
o_star   = 3.716132203115754

#Larvae ---------------------
#sigma_0  = 1.458299960257398

#Spawning -------------------
sigma_0  = 1.388505133695817

sigma_K  = 3.255065398014956

#Larvae ---------------------
#T_star_1 = 28.72407840883895

# Spawning ------------------
T_star_1 = 31.13068425631411

T_star_K = 27.54999999968445
bT       = 2.564600347374714

In [6]:
fh.sigmaStar(1.458299960257398, sigma_K)

array([3.2550654])

In [7]:
fh.sigmaStar(1.388505133695817, sigma_K)

array([3.2550654])

In [8]:
fh.tStar(28.72407840883895, T_star_K, bT)

array([27.55])

In [9]:
fh.tStar(31.13068425631411, T_star_K, bT)

array([27.55])

In [6]:
%%time
result_ha = fh.Ha(fpc=fpc, gamma=gamma, o_star=o_star, sigma_0=sigma_0, sigma_K=sigma_K, T_star_1=T_star_1, T_star_K=T_star_K, bT=bT)

Temperature
Oxygen
Forage
Wall time: 44.3 s


In [8]:
result_ha.shape

(1, 264, 480, 808)

In [9]:
result_ha.nbytes

409559040

In [10]:
result_ha.dtype

dtype('float32')

In [11]:
result_atan_ha = fh.scaling(result_ha[0,:,:,:])

----
# SAVE

In [12]:
coords = xr.open_dataset(filepath_directory+"MFCat_po_025x30_temperature_L1_1998_2019.nc").coords

In [13]:
da = xr.DataArray(
    data=result_atan_ha,
    dims=["time", "lat", "lon"],
    coords=dict(
        lon=coords['lon'],
        lat=coords['lat'],
        time=coords['time']),
    attrs=dict(description="Ha_oldest"))

In [14]:
da

<xarray.DataArray (time: 264, lat: 480, lon: 808)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)
Coordinates:
  * lon      (lon) float32 88.5 88.75 89.0 89.25 ... 289.5 289.8 290.0 290.2
  * lat      (lat) float32 65.25 65.0 64.75 64.5 ... -53.75 -54.0 -54.25 -54.5
  * time     (time) datetime64[ns] 1998-01-15T12:00:00 ... 2019-12-16T12:00:00
Attributes:
    description:  Ha_oldest

In [15]:
ds = da.to_dataset(name='Ha_oldest')

In [16]:
ds

<xarray.Dataset>
Dimensions:    (lat: 480, lon: 808, time: 264)
Coordinates:
  * lon        (lon) float32 88.5 88.75 89.0 89.25 ... 289.5 289.8 290.0 290.2
  * lat        (lat) float32 65.25 65.0 64.75 64.5 ... -53.75 -54.0 -54.25 -54.5
  * time       (time) datetime64[ns] 1998-01-15T12:00:00 ... 2019-12-16T12:00:00
Data variables:
    Ha_oldest  (time, lat, lon) float32 nan nan nan nan nan ... nan nan nan nan

In [17]:
ds.to_netcdf("SPAWNING_V2_computed_scaled_po_025x30_Ha_oldest.nc")

----
----

In [17]:
real_oldest = xr.open_dataset("C:\\Users\\julesl\\Documents (Local)\\JupyterLabWorkspace\\\Data\\Concat\\PO\\output\\MFCat_SEAPODYM_skj_Ha_oldest_025x30d_1998_2019.nc")

In [18]:
real_oldest.load()

<xarray.Dataset>
Dimensions:        (lat: 480, lon: 808, time: 264)
Coordinates:
  * lat            (lat) float32 65.25 65.0 64.75 64.5 ... -54.0 -54.25 -54.5
  * lon            (lon) float32 88.5 88.75 89.0 89.25 ... 289.8 290.0 290.2
  * time           (time) datetime64[ns] 1998-01-15T12:00:00 ... 2019-12-16T1...
Data variables:
    ha_oldest_skj  (time, lat, lon) float32 nan nan nan nan ... nan nan nan nan
Attributes:
    history:  Created on 2021-05-25

In [19]:
computed = ds['Ha_oldest'].data
computed.shape

(264, 480, 808)

In [20]:
observed = real_oldest['ha_oldest_skj'].data
observed.shape

(264, 480, 808)

----
# COMPARISON
#### COMPUTED

In [21]:
np.sum(np.isfinite(computed))

56574672

In [22]:
np.sum(np.isnan(computed))

45815088

In [23]:
computed.size

102389760

In [24]:
# computed
56023440 + 46366320

102389760

#### OBSERVED

In [25]:
np.sum(np.isfinite(observed))

56574672

In [26]:
np.sum(np.isnan(observed))

45815088

In [27]:
observed.size

102389760

In [28]:
# observed
56574672 + 45815088

102389760

----
# Mean Square Error

In [30]:
np.nanmean((observed - computed)**2)

0.046392247

In [31]:
np.nanmean(observed)

0.5049858

In [32]:
np.nanmean(computed)

0.3800318